<a href="https://colab.research.google.com/github/sekihiro/Colabo/blob/master/bert_pre_trained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【PyTorch】BERTの使い方 - 日本語pre-trained models
- 事前学習した日本語pre-trained modelsの精度を確認します。
- 今回はMasked Language Modelの精度を確認します。
- Masked Language Modelを簡単に説明すると、文の中のある単語をマスクしておき、そのマスクされた単語を予測するというものです。
- https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## defaultでインストールされていないライブラリを入れる
- MeCab と transformers(旧名：pytorch-pretrained-BERT)

In [0]:
# MeCab
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
# transformers(旧名：pytorch-pretrained-BERT)
!pip install transformers

In [5]:
import os
import sys
import pprint
import random
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM, BertForSequenceClassification

In [6]:
# 再現性確保
def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    return seed

# 再現性確保
ret = seed_everything(1234)
print('seed : {}'.format(ret))

# select device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device : {}'.format(device))

seed : 1234
device : cuda


## tokenizer と model の取得
- 東北大学 乾・鈴木研究室の作成・公開されたBERTモデル
- 公開されているモデルは４種類あるが、bert-base-japanese-whole-word-masking を使うのが良い
- 日本語Wikipediaを用いて学習
- okenizerはMeCab + WordPiece（character tokenizationもある）
- max sequence lengthは512
- https://qiita.com/nekoumei/items/7b911c61324f16c43e7e

### transformers で定義されているクラス
- http://kento1109.hatenablog.com/entry/2019/08/20/161936
- BertForMaskedLM : 単語を出力するためのクラス
- BertForSequenceClassification : 分類問題のためのクラス

In [7]:
BASE_PATH = "/content/drive/My Drive/git/"
MODEL_PATH = BASE_PATH + 'model/bert/base-japanese-whole-word-masking/'

if os.path.isfile(MODEL_PATH + 'vocab.txt'):
    print('loading bert-pytorch_tokenizer ..... ')
    tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_PATH)
else:
    print('downloading bert-pytorch_tokenizer ..... ')
    tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
    tokenizer.save_pretrained(MODEL_PATH)

if os.path.isfile(MODEL_PATH + 'pytorch_model.bin'):
    print('loading bert-pytorch_model ..... ')
    model = BertForMaskedLM.from_pretrained(MODEL_PATH)
else:
    print('downloading bert-pytorch_model ..... ')
    model = BertForMaskedLM.from_pretrained('bert-base-japanese-whole-word-masking')
    model.save_pretrained(MODEL_PATH)

model.to(device)

loading bert-pytorch_tokenizer ..... 
loading bert-pytorch_model ..... 


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

#### *funcation：* MASK部分を予測する 

In [0]:
def pred_mask(token_list):

    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(token_list)
    # [571, 12, 4, 5, 608, 11, 2867, 8]

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to(device)
    # tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

    # Predict
    model.eval()
    with torch.no_grad(): # 必要のない計算を停止 パラメータの保存を止める
        outputs = model(tokens_tensor)
        ##### outputs ===> [1, 8, 32000]
        #pprint.pprint(outputs)
        #pprint.pprint(outputs[0])
        #pprint.pprint(outputs[0][0])
        #pprint.pprint(outputs[0][0][masked_index]) # [MASK]の部分の予測値を得る
        predictions = outputs[0][0][masked_index].topk(5) # 予測結果の上位N件を抽出(値とインデックスを得る)
        #print('----------')
        #pprint.pprint(predictions)

        for i, index_t in enumerate(predictions.indices): # インデックスのみ採用
            index = index_t.item() # tensor -> int
            token = tokenizer.convert_ids_to_tokens([index])[0]
            print(i, token)

    return

#### MASK部分を求める

In [18]:
# Tokenize input
#text = 'テレビでサッカーの試合を見る'
text = '僕は友達とサッカーをすることが好きだ'
print(text)
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 4
tmp = ''
for i, buf in enumerate(tokenized_text):
    if i == masked_index:
        tmp = tmp + '[MASK]'
    else:
        tmp = tmp + tokenized_text[i]
#print(tmp)
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)
print('---------------')
pred_mask(tokenized_text)

print('\n')
tokenized_text.insert(0, '[CLS]')
tokenized_text.append('[SEP]')
print(tokenized_text)
print('---------------')
pred_mask(tokenized_text)

僕は友達とサッカーをすることが好きだ
['僕', 'は', '友達', 'と', '[MASK]', 'を', 'する', 'こと', 'が', '好き', 'だ']
---------------
0 話
1 仕事
2 ゲーム
3 食事
4 ゴルフ


['[CLS]', '僕', 'は', '友達', 'と', '[MASK]', 'を', 'する', 'こと', 'が', '好き', 'だ', '[SEP]']
---------------
0 と
1 で
2 とも
3 の
4 とともに
